# Hopper study

## Return map

assume a 1-D hopping robot that has rest leg length $l_0$, spring constant, $k$, and mass $m$. When the robot is in the air the leg is pre-compressed by an amount $
\Delta L$. When the foot collides with the ground a small amount of center-of-mass velocity is lost $y^+ = \epsilon y^-$ such that the superscript $+$ denotes immediately after the collision event and $-$ immediately before. The coefficient of restitution is bounded by $0 < \epsilon < 1$.

* Derive the apex return map for a hopping robot with leg pre-compression as discussed in class. The apex is the peak height of the hopping robot, thus when $\dot{y} = 0$ in the flight phase. This can be derived by composing the following maps 
\begin{align}
    \mathcal{R} = \mathcal{F}_a \circ \mathcal{S} \circ \mathcal{C} \circ \mathcal{F}_d
\end{align}
where $\mathcal{F}_d$ is the descent map from apex to beginning of stance, $\mathcal{C}$ is the instantaneous ground collision map, $\mathcal{S}$ is the stance map from beginning of stance to end, and $\mathcal{F}_a$ is the ascent map from end of stance to apex. The $\circ$ symbol indicates function composition (i.e. $\mathcal{D} = g \circ h = g(h(x))$. Remember, a map is just a function that returns the end state for a given initial state. 

* Compute the steady-state hopping height of the robot as a function of system parameters. 


## Derivation

To derive the apex map we write out the input/output relationships for each of the regimes as follows: 

* The ascent map: $\mathcal{F_a}$

$$ m g y_f = \frac{1}{2} m \dot{y}_0^2$$ 

* The stance map: $\mathcal{S_a}$

$$ \frac{1}{2} m \dot{y}_f^2 = \frac{1}{2} m \dot{y}_0^2 + \frac{1}{2} k \Delta l^2 - mg \Delta l  $$

* The contact map: $\mathcal{C}$

$$ \dot{y}_f = \epsilon \dot{y}_0 \longrightarrow \frac{1}{2} m \dot{y}_f^2 = \epsilon^2 \frac{1}{2} m \dot{y}_0^2$$

* The descent map: $\mathcal{d}$

$$ \frac{1}{2} m \dot{y}_f^2 - mg \Delta l = mg y_0 \longrightarrow \frac{1}{2} m \dot{y}_f^2 = mg (y_0 + \Delta l) $$

Plugging these functions in in the order described above and simplifying we get

$$y_{n+1} = \epsilon^2 y_{n} + \Delta l (\epsilon^2 - 1) + \frac{k}{2 m g} \Delta l^2$$

## Fiexed point

Substituting in the fixed point $y_n = y_{n+1} = y^*$ we get 

$$y^*(1 - \epsilon^2) =  \Delta l (\epsilon^2 - 1) + \frac{k}{2 m g} \Delta l^2$$

which yields 

\begin{align}
y^* &=  \Delta l \frac{(\epsilon^2 - 1)}{(1 - \epsilon^2)} + \frac{k}{2 m g} \frac{\Delta l^2}{(1 - \epsilon^2)} \\
 &= \frac{k}{2 m g} \frac{\Delta l^2}{(1 - \epsilon^2)} - \Delta l
\end{align}



## Part 3

In [7]:
import numpy as np
from scipy.integrate import solve_ivp

import matplotlib as mpl
mpl.use('Qt5Agg')

import matplotlib.pyplot as plt
plt.ion()


import warnings
warnings.filterwarnings('ignore')

## Continuous simulation of the hopper

The equations of motion are as follows: 
1. In the flight phase 
\begin{align}
\begin{bmatrix}
\dot{y_1} \\
\dot{y_2}
\end{bmatrix} = 
\begin{bmatrix}
y_2 \\
-g
\end{bmatrix}
\end{align}

2. In the stance phase 
\begin{align}
\begin{bmatrix}
\dot{y_1} \\
\dot{y_2}
\end{bmatrix} = 
\begin{bmatrix}
y_2 \\
-\frac{k}{m} (y_1 - l_0) - g
\end{bmatrix}
\end{align}

In [8]:
g = 9.8

def stance(t, state, k, l_0, m, b):
    y1, y2 = state
    next_state = [y2, 
                 -(k/m)*(y1 - l_0) - g - (b/m)*y2]
    return next_state

def end_stance_event(t, state, l_0):
    y1, y2 = state
    return y1 - l_0
end_stance_event.terminal = True

def flight(t, state):
    y1, y2 = state
    next_state = [y2, 
                  -g]
    return next_state

def apex_flight_event(t, state):
    y, ydot = state
    return ydot
apex_flight_event.terminal = True

def end_flight_event(t, state, l_0, delta_l):
    y1, y2 = state
    return y1 - (l_0 - delta_l) # this handles the pre-compression
end_flight_event.terminal = True

def collision(t, state, epsilon):
    y1, y2 = state
    next_state = [y1, 
                  epsilon*y2]
    return next_state


To simulate a hopping motion we need to compose together these sequences in a single function. **Important, we will make the assumption that the hopper starts at the apex of the flight phase**

## Make a function that composes the hybrid dynamics

In [9]:
def simulate_hop(t, state, delta_l, k, m, l_0, epsilon, t0, b):
    ## This is a reasonably long time so that each phase finishes before integration stops
    long_time = 100
    time_outs = np.linspace(0, long_time, int(long_time/0.001))

    ##### Solve the flight descent phase    
    end_flight_lambda = lambda t, state: end_flight_event(t, state, l_0, delta_l)
    end_flight_lambda.terminal = True

    sol = solve_ivp(lambda t, state: flight(t, state), 
                [t0, t0 + long_time], state, 
                events = end_flight_lambda, 
                t_eval= t0 + time_outs,
                rtol=1e-10, atol = 1e-10, 
                dense_output = True, 
                method = 'RK45', 
                max_step = 1)
    land_state = sol.y[:,-1]
    
    ##### Grab the solutions trajectory
    out_state = sol.y
    time = sol.t
    t0 = sol.t[-1]
    
    ##### Map landing state to beginning of stance state
    stance_state = collision(0, land_state, epsilon)
    
    ##### Integrate the stance state
    end_stance_lambda = lambda t, state: end_stance_event(t, state, l_0)
    end_stance_lambda.terminal = True
    
    sol = solve_ivp(lambda t, state: stance(t, state, k, l_0, m, b), 
            [t0, t0 + long_time], stance_state, 
            events = end_stance_lambda, 
            t_eval= t0 + time_outs,
            rtol=1e-10, atol = 1e-10, 
            dense_output = True, 
            method = 'RK45', 
            max_step = 1)

    ##### Grab the solutions trajectory
    out_state = np.hstack((out_state, sol.y))
    time = np.hstack((time, sol.t))   
    ascent_state = sol.y[:,-1]
    t0 = sol.t[-1]
    
    ##### Solve the flight ascent phase    
    end_flight_lambda = lambda t, state: apex_flight_event(t, state)
    end_flight_lambda.terminal = True

    sol = solve_ivp(lambda t, state: flight(t, state), 
                [t0, t0 + long_time], ascent_state, 
                events = end_flight_lambda, 
                t_eval= t0 + time_outs,
                rtol=1e-10, atol = 1e-10, 
                dense_output = True, 
                method = 'RK45', 
                max_step = 1)

    out_state = np.hstack((out_state, sol.y))
    time = np.hstack((time, sol.t))   
    apex_state = sol.y[0,-1]
    
    return time, out_state, apex_state
    

Let's run one simulation starting from an apex height of 10

In [10]:
k = 50
m = 5
l_0 = 5
delta_l = 1
epsilon = 0.9
b = 0
initial_state = [l_0 + 1, 0]

time, out_state, apex_state = simulate_hop(0, initial_state, delta_l, k, m, l_0, epsilon, 0, b)

plt.figure(figsize = (10, 8))
plt.clf()
plt.subplot(2,1,1)
plt.plot(time, out_state[0,:])
plt.xlabel('Time (s)')
plt.ylabel('Height')

plt.subplot(2,1,2)
plt.plot(out_state[0,:], out_state[1,:])
# plt.gca().set_aspect('equal', 'box')
plt.xlabel('Vertical location')
plt.ylabel('Vertical velocity')

Text(0, 0.5, 'Vertical velocity')

Let's simulate a number of hops now!

### Calculate numerical return map

In [17]:
k = 50
m = 5
l_0 = 5
delta_l = 1
epsilon = .9
b = 0
initial_state = [l_0+1, 0]
t0 = 0

time, out_state, apex_state = simulate_hop(0, initial_state, delta_l, k, m, l_0, epsilon, 0, b)
initial_state = out_state[:, -1]
t0 = time[-1]

for kk in range(10):
    
    t, st, at = simulate_hop(0, initial_state, delta_l, k, m, l_0, epsilon, t0, b)
        
    time = np.hstack((time, t))
    out_state = np.hstack((out_state, st))
    apex_state = np.hstack((apex_state, at))
    
    initial_state = out_state[:, -1]
    t0 = time[-1]
    print(kk)

0
1
2
3
4
5
6
7
8
9


### Hand derived return map

In [18]:
def analytical_return_map(y_in, epsilon, delta_l, k, m, l_0):
    g = 9.8
    return epsilon**2*y_in + delta_l*(epsilon**2 - 1) + (k/m)/(2*g)*delta_l**2

y_in = np.linspace(0,8,100)
y_out = analytical_return_map(y_in, epsilon, delta_l, k, m, l_0)

In [19]:
    
plt.figure(figsize = (14, 6))
plt.clf()
plt.subplot(2,2,1)
plt.plot(time, out_state[0,:])
plt.xlabel('Time')
plt.ylabel('Height')

plt.subplot(2,2,2)
plt.plot(apex_state[:-2] - l_0, apex_state[1:-1] - l_0, 'o')
plt.plot(y_in, y_out)

plt.plot([0,10],[0,10], '-')
plt.xlabel('$y_{n} - l_0$')
plt.ylabel('$y_{n+1} - l_0$')
plt.title('Numerical (points) and analytical (orange line) return map')

plt.subplot(2,2,(3,4))
plt.plot(out_state[0,:], out_state[1,:])
plt.xlabel('Vertical location')
plt.ylabel('Vertical velocity')

Text(0, 0.5, 'Vertical velocity')